### Webスクレイピング
---
URLでアクセスできるものであれば、取得できる


#### requests.get()メソッド
---


In [2]:
import requests


# 取得したいURLを書きます
url = "http://docs.pyq.jp/_static/assets/scraping/test1.html"

# HTTPリクエストを送信してHTMLを取得します
response = requests.get(url)
response.encoding = response.apparent_encoding

# 取得したHTMLを表示します
print(response.text)

<!DOCTYPE html>
<html lang="ja">
  <head>
    <meta charset="utf-8">
    <title>最初のHTML</title>
  </head>
  <body>
    <h1>最初のHTML</h1>
    <p>スクレイピングテスト</p>
  </body>
</html>



In [3]:
import requests

# 取得したいCSVのURLを書きます
url = "http://docs.pyq.jp/_static/assets/scraping/test2.csv"

# HTTPリクエストを送信してCSVを取得します
response = requests.get(url)
response.encoding = response.apparent_encoding

# 取得したCSVを表示します
print(response.text)

Spam,Ham,Egg
Spam2,Ham2,Egg2
Spam3,Ham3,Egg3



#### 連続して同じサイトにアクセスするときは間隔を最低1秒空ける
---
対象となるサイトに迷惑をかけないため

索引次へ |前へ |PyQ 1.0 ドキュメント »コラム »
スクレイピング、クローリングする時の注意点
スクレイピング、クローリングのプログラムを作成して実行する場合には、いくつか注意すべき点があります。

公開されている情報とはいえ、スクレイピングとクローリングは自分が管理していないWebサーバーにアクセスして情報を取得します。

そもそもそのサイトにWebブラウザーではなくプログラムからアクセスしても良いのか、Webページに記載している内容をデータ化しても良いのかといった利用制限について確認する必要があります。

また複数のページから情報を抜き出す場合に、連続アクセスするとWebサーバーに負荷をかけてることになります。

その場合クロールされる側の迷惑になったり、DoS攻撃（サーバーに過剰な負荷をかけてサービスを妨害する攻撃）とみなされることがあります。

以下に注意すべき項目についていくつか解説します

著作権と利用規約
Webページ上の情報の著作権や利用規約を確認します。Webページ上の情報を抜き出して再利用することを利用規約で禁じているサイトもあります。そういったサイトの情報をスクレイピングで抜き出した場合、法的に訴えられることもあるため、注意が必要です。

サイトマップ
サイトマップとは、サイト運営者がクローラーに対してクロールして欲しいURLを列挙するXMLファイルです。闇雲にサイトを巡回してクロールするよりも、効率よく且つ正確にクロールできます。

サイトマップXMLの配置場所は、後述するrobots.txtで提示します。クローラー側はこの情報を読み取ってサイトマップXMLを解析しクロール対象のURLを取得します。

サイトマップの仕様については下記URLを参照してください。

https://www.sitemaps.org/ja/

robots.txtによる指示
robots.txtというファイルには、ロボットに対してどのURLにアクセスして良いか、してはいけないかが書いてあります。

ロボットとは、ここではGoogleクローラーのようなWebページを収集するプログラムのことを指します

robots.txtは慣習的にサイトのURLの直下に置かれます。たとえば下記のようなrobots.txtがあったとして簡単に読み方を説明します。

User-agent: *
Crawl-delay: 5
Disallow: /ng
Allow: /ok

Sitemap: http://example.com/sitemap.xml
User-agent ... 対象となるクローラーの種類
Disallow ... クロールを禁止するパス
Allow ... クロールを許可するパス
Crawl-delay .. クロールする間隔（秒）
Sitemap ... サイトマップXMLのURL
User-agent: * となっているので、全てのクローラーに対しての指示です。 /ngへのクロールは禁止するが、/okなどのURLはクロールを許可するという指示となっています。

robots.txtについてより詳しくはGoogleのドキュメントが参考になるでしょう。

https://developers.google.com/webmasters/control-crawl-index/docs/robots_txt

Pythonでrobots.txtを解析する際は、urlib.robotparserが利用できます。

>>> import urllib.robotparser
>>> rp = urllib.robotparser.RobotFileParser()
>>> rp.set_url("http://www.musi-cal.com/robots.txt")
>>> rp.read()
>>> rrate = rp.request_rate("*")
>>> rrate.requests
3
>>> rrate.seconds
20
>>> rp.crawl_delay("*")
6
>>> rp.can_fetch("*", "http://www.musi-cal.com/cgi-bin/search?city=San+Francisco")
False
>>> rp.can_fetch("*", "http://www.musi-cal.com/")
True
Webサイトへの負荷
クローラーはWebサーバーにプログラムでアクセスして、HTMLなどの情報を取得します。

クローラーの作り方によっては、同時に複数のURLに接続してデータを取得したり、短時間で連続してURLからデータを取得できます。 しかし、このようなかたちでWebサーバーにアクセスすると、Webサーバーに負荷をかける場合があります。Webサーバーに負荷をかけると、他のユーザーがそのWebサイトを参照できなかったり、ひどい場合はサーバーが落ちてしまう場合もあります。

そのような迷惑をかけないためにも、クローラーは間隔をあけてWebサーバーにアクセスするといった対応が必要となります。最低1秒以上は間隔をあけるようにしましょう。

連絡先の明示
クローラーを開発する際は連絡先を明示しましょう、何かしら問題がある時にサーバー管理者は連絡できるので問題解決しやすくなります。

連絡先を明示する手段としては、クローラーのUser-Agentヘッダーに連絡先のURLやメールアドレスを書く方法が一般的です。

User-Agentヘッダーの情報がサーバーにログとして残っていれば、サーバー管理者は、そのログを確認して連絡先を確認できます。

たとえばGoogleのクローラーは「http://www.google.com/bot.html」というURLをUser-Agentヘッダーに含めることで、連絡先を明示しています。

Googleのユーザーエージェント例:

Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)
Pythonのrequestsというライブラリーでの、User-Agentヘッダーの設定方法を記載します。ご利用のライブラリーによって設定の仕方が違うので、適宜そのライブラリーのドキュメントを参照してください。

>>> import requests
>>> url = 'http://example.com'
>>> headers = {
...  'User-Agent': 'PyQ Example Crawler/1.0.0 (http://pyq.jp)'
... }
>>> response = requests.get(url, headers=headers)
>>> ressponse.request.headers
{'User-Agent': 'PyQ Example Crawler/1.0.0 (http://pyq.jp)', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
索引次へ |前へ |PyQ 1.0 ドキュメント »コラム »
© Copyright 2017, BeProud. このドキュメントは Sphinx 1.5.2 で生成しました。